In [43]:
# imports
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import Data.data_provider as dp
import core as cal
from estimators.IR_RF_estimator import IR_RF
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier


In [44]:
# params
calib_methods = ["RF"] #cal.calib_methods.copy() 
metrics = ["acc", "tce"]#cal.metrics.copy()

plot = True
 
params = {
    "runs": 1,
    "n_tree": 100, 
    "depth": [5, 6, 7],
    "data_size": 10000,
    "n_features": 2,
    "oob": False,
    "test_split": 0.3,
    "calib_split": 0.1
}

In [45]:
data_list = []

X, y, tp = dp.make_classification_gaussian_with_true_prob(params["data_size"], params["n_features"], 
                                                          class1_mean_min=0, 
                                                          class1_mean_max=1,

                                                          class2_mean_min=1, 
                                                          class2_mean_max=3, 

                                                          seed=0)

# plot data
# plt.scatter(X[:,0], X[:,1], c=y)
# plt.show()

In [46]:
calib_results_dict = {}
for exp_param in params["depth"]:
    data_dict = {} # results for each data set will be saved in here.
    # Data
    exp_data_name = str(exp_param)
    data_list.append(exp_data_name)

    for seed in range(params["runs"]): # running the same dataset multiple times
        # split the data
        data = cal.split_train_calib_test(exp_data_name, X, y, params["test_split"], params["calib_split"], seed, tp)

        # train models
        models = {}

        dt = DecisionTreeClassifier().fit(data["x_train"], data["y_train"])
        models["dt"] = dt

        irrf = IR_RF(n_estimators=params["n_tree"], max_depth=exp_param, oob_score=params["oob"], random_state=seed).fit(data["x_train"], data["y_train"])
        models["RF"] = irrf

        irrf_l = IR_RF(n_estimators=params["n_tree"], max_depth=exp_param, oob_score=params["oob"], random_state=seed).fit(data["x_train"], data["y_train"])
        models["RF_l"] = irrf_l

        lr = LogisticRegression(random_state=seed).fit(data["x_train"], data["y_train"])
        models["LR"] = lr

        svm = SVC(probability=True, random_state=seed).fit(data["x_train"], data["y_train"])
        models["SVM"] = svm


        # calibration
        res = cal.model_calibration(models, data, metrics) # res is a dict with all the metrics results as well as RF probs and every calibration method decision for every test data point
        
        data_dict = cal.update_runs(data_dict, res) # calib results for every run for the same dataset is aggregated in data_dict (ex. acc of every run as an array)
        
        if plot:
            cal.plot_probs(exp_data_name, res, data, models, seed) 

        calib_results_dict.update(data_dict) # merge results of all datasets together


In [47]:
tables = cal.mean_and_ranking_table(calib_results_dict, metrics, models, data_list, mean_and_rank=False)

In [48]:
tables["acc"]

,dt,RF,RF_l,LR,SVM
Data,,,,,
5,0.716000,0.791667,0.792333,0.795,0.793
6,0.712667,0.792333,0.792000,0.795,0.793
7,0.714000,0.792000,0.791333,0.795,0.793


In [49]:
tables["tce"]

,dt,RF,RF_l,LR,SVM
Data,,,,,
5,0.138688,0.002595,0.002649,0.000391,0.007113
6,0.139325,0.002363,0.002345,0.000391,0.007113
7,0.139063,0.002929,0.002720,0.000391,0.007113
